# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Para isso, vamos criar uma automação web:

- Usaremos o selenium


In [25]:
# Passo 1 - Entrar na internet
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()

# Passo 2 - Pegar cotação do Dolar

# entrar no google

navegador.get('https://www.google.com.br/')

# pesquisar 'cotação dolar'

navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')

navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegar o numero que aparece no resultado

cotacao_dolar = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')



# Passo 3 - Pegar contação do Euro

# pesquisar 'cotação euro'

navegador.find_element_by_xpath('//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input').clear()

navegador.find_element_by_xpath('//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input').send_keys('cotação euro')

navegador.find_element_by_xpath('//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input').send_keys(Keys.ENTER)

# pegar o numero que aparece no resultado

cotacao_euro = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')



# passo 4 - Pegar cotação do Ouro

# entrar no site melhor câmbio hoje

navegador.get('https://www.melhorcambio.com/ouro-hoje')


# pegar numero que aparece no resultado

cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')
navegador.close()


print(f'Cotação Dolar: {cotacao_dolar}')
print(f'Cotação Euro: {cotacao_euro}')
print(f'Cotação Ouro: {cotacao_ouro}')

Cotação Dolar: 5.2463
Cotação Euro: 6.154959160000001
Cotação Ouro: 295.59


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [26]:
import pandas as pd

tabela = pd.read_excel('Produtos (1).xlsx') 
tabela

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [29]:
# Atualizar a contação
# tabela.loc[Linha, Coluna]

# Aonde na 'Moeda' == 'Dólar'
tabela.loc[tabela['Moeda']=='Dólar', 'Cotação'] = float(cotacao_dolar)

# Aonde na 'Moeda' == 'Euro'
tabela.loc[tabela['Moeda']=='Euro', 'Cotação'] = float(cotacao_euro)

# Aonde na 'Moeda' == 'Ouro'
tabela.loc[tabela['Moeda']=='Ouro', 'Cotação'] = float(cotacao_ouro)


# Atualizar o preço de compra: preço original x cotacçao

tabela['Preço de Compra'] = tabela['Preço  Original'] * tabela['Cotação'].round(1)


### Agora vamos exportar a nova base de preços atualizada

In [28]:
tabela.to_excel('Produtos_Novo.xlsx', index=False)